In [1]:
from Modele_db.modele_db import *
from Pianki.Analiza_pianek.funkcje_analizy_pianek import *

# pd.set_option()

In [2]:
Podsumowanie_paczek_i_Pw("")

OBIĘTOSC PACZEK (Z OWATAMI)
14/01: 150m3
15/01: 136m3
16/01: 114m3
17/01: 161m3
WST: 9m3
Objetość wszystkich paczek: 570m3
PW  spakowano: 0M3


In [3]:
br = Braki(WOLNE="SALDO")[0] #USUNAC PROBLEM Z DUBLUJACYMI SIE ZAMOWIANIAMI(JEDNO DOSTARCZONE DRUGIE W DRODZE TWORZĄ SIĘ DWIE POZYCJE)
br.sort_values(by=["GRUPA", "OPIS"])
# br.to_excel("BRAKI.xlsx")

,OPIS,WOLNE_SALDO,SALDO,PACZKA,DATA_KOMPLETACJI,ZAMOWIONE,UWAGI,GRUPA
3,MYSTIC N120,-1.0,1.0,17/01,2024-06-13,0.0,...,1
4,STONE WN150x140 LEN,-2.0,7.0,17/01,2024-06-13,0.0,...,1
6,UNO P48x48,-6.0,0.0,14/01,2024-05-22,0.0,...,1
0,AVANT P64x64,-2.0,0.0,17/01,2024-06-13,10.0,"PIANKI ZAMOWIONE P, , ,",2
5,"WIL. 2,5 Z BOCZ - OWA",-2.0,1.0,14/01,2024-05-22,15.0,"PIANKI ZAMOWIONE O, , ,",2
2,HORIZON P62X62,-3.0,1.0,14/01,2024-05-22,10.0,CZEKA NA SPAKOWANIE,4
1,HORIZON P82x82,-1.0,7.0,16/01,2024-06-06,20.0,CZEKA NA SPAKOWANIE,4


In [4]:
Zagrozone(WOLNE="SALDO")
zgr = Zagrozone(False,WOLNE="SALDO")
# zgr
# zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)]
# zgr[(zgr.SALDO < zgr.MIN)]
zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)&(zgr.SALDO < zgr.MIN)]

WOLNE PONIZEJ MIN: 58 POZYCJE
WOLNE PONIZEJ MIN NIE ZAMOWIONE: 21 POZYCJE
SALDO PONIZEJ MIN: 24 POZYCJE
SALDO PONIZEJ MIN NIE ZAMOWIONE: 10 POZYCJE


,OPIS,ZAMOWIONE,CZEKA_NA_SPAKOWANIE,SALDO,MIN,WOLNE_SALDO
0,AVANT [1],0.0,0.0,0.0,2,0.0
13,AVANT 2][,0.0,0.0,5.0,7,1.0
82,DUO P82x82,0.0,0.0,0.0,1,0.0
94,"EXTREME 1,5s",0.0,0.0,0.0,8,0.0
97,"EXTREME [1,5s",0.0,0.0,0.0,8,0.0
107,EXTREME P80x105,0.0,0.0,0.0,8,0.0
197,RITZ 4,0.0,0.0,0.0,4,0.0
202,RITZ Z60,0.0,0.0,0.0,4,0.0
262,LENOX [1H],0.0,0.0,64.0,65,25.0
266,UNO P63x63,0.0,0.0,10.0,15,4.0


In [5]:
Ogolna_analiza_objetosci()
print("---\nZAPEŁNIENIE MAGAZYNU")
Ogolna_analiza_objetosci("podsum_prc")

MAX_obj:                    3190
ZAMOWIONE_obj:              366
CZEKA_NA_SPAKOWANIE_obj:    162
SALDO_obj:                  2735
WOLNE_obj:                  2165
WOLNE_NIE_SPAK_obj:         2328
DO_ZAM_obj:                 663
---
ZAPEŁNIENIE MAGAZYNU
SALDO / MAX: 85.8%
WOLNE / MAX: 67.9%
WOLNE_NIE_SPAK / MAX: 73.0%
ZAPEŁNIENIE MAG PO ZDJECIU 4 PACZEK RAZEM Z ZAMOWIENIAMI: 84.4%


In [6]:
#IMPORT ANAMLIZY PIANEK
from Pianki.Analiza_pianek.instrukcje_zamawiana import instrukcja_zamawiania_pianpol as izp, instrukcja_zamawiania_ciech as izc
from Pianki.Analiza_pianek.Podsumowanie_analizy_pianek import Podsumowanie_analizy_pianek

arp = Podsumowanie_analizy_pianek(izp)
# arc = Podsumowanie_analizy_pianek(izc)



In [7]:
arp

OBJ_CIECH       30.74
OBJ_VITA        52.79
OBJ_PIANPOL    339.16
RAZEM          422.69

In [8]:
arp > 20

,GRUPA,MODEL,BRYL_DO_ZAMOWIENIA,WSPL_DO_ZAM,OBJ_CIECH,OBJ_VITA,OBJ_PIANPOL,*WSPL_ZAP_WOLNE,*WSPL_ZAP_ZAM
1,1,ELIXIR,89.0,27.0,0.00,0.00,43.85,74.0,74.0
2,1,STONE,236.0,22.0,0.00,0.00,54.78,83.0,83.0
3,1,HORIZON,271.0,21.0,0.00,52.79,11.76,47.0,79.0
9,2,REVERSO,196.0,26.0,0.00,0.00,45.11,67.0,74.0
6,2,MAXWELL,31.0,21.0,0.00,0.00,13.13,85.0,85.0
16,3,SPECTRA,42.0,37.0,18.48,0.00,0.00,67.0,67.0
13,3,ONYX,55.0,34.0,0.00,0.00,19.16,52.0,74.0
14,3,RITZ,26.0,28.0,5.17,0.00,0.00,122.0,122.0
21,4,UNO,62.0,78.0,0.00,0.00,2.15,35.0,35.0
20,4,LENOX,163.0,66.0,0.00,0.00,16.98,34.0,34.0


In [13]:
arp["AVANT"].ar_skr.columns

Index(['OPIS', 'ZAMOWIONE', 'CZESIOWO_DOSTARCZONE', 'CZEKA_NA_SPAKOWANIE',
       'SALDO', 'SALDO_Z_NIE_SPAK', 'MIN', 'SUMA_ZLEC', 'WOLNE_SALDO',
       'WOLNE_NIE_SPAK'],
      dtype='object')

In [10]:
"sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db"



'sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db'

In [11]:
# Zapis_danych_do_Archiwum(xxx, analiza, arp._Tabela_podsumowania_analizy, "sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db")